In [10]:
# Recomendado: Python 3.10+ en kernel local (VS Code + extensión Jupyter).
# Crea un entorno virtual si quieres aislar dependencias:
#   python -m venv .venv
#   .venv\Scripts\activate   # Windows
#   source .venv/bin/activate  # macOS/Linux

In [19]:
%pip install -q pandas numpy requests pyarrow matplotlib python-dotenv


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: C:\Users\crist\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [12]:
import os
from pathlib import Path
import math
import time
import re
import json
import requests
import pandas as pd
import numpy as np
from dotenv import load_dotenv

pd.set_option("display.max_columns", 100)
pd.set_option("display.width", 120)

In [13]:
# URL base de la API (Socrata SODA)
API_URL = "https://www.datos.gov.co/resource/xfif-myr2.json"

# Opcional: usa un token de app Socrata para evitar límites estrictos (si tienes cuenta).
# Crea un archivo .env con:
#   SODA_APP_TOKEN=tu_token
load_dotenv()
SODA_APP_TOKEN = os.getenv("SODA_APP_TOKEN")

# Estructura local del repo (sugerida)
DATA_DIR = Path("data")
RAW_DIR = DATA_DIR / "raw"
PROC_DIR = DATA_DIR / "processed"
FIG_DIR = Path("figures")
for d in [RAW_DIR, PROC_DIR, FIG_DIR]:
    d.mkdir(parents=True, exist_ok=True)

RAW_FILE = RAW_DIR / "xfif_myr2_raw.parquet"
PROC_FILE = PROC_DIR / "xfif_myr2_clean.parquet"

print("Rutas creadas ✅")


Rutas creadas ✅


In [14]:
def fetch_socrata_dataset(
    api_url: str,
    select: str | None = None,
    where: str | None = None,
    order: str | None = None,
    limit: int = 50000,
    max_rows: int = 100_000,
    app_token: str | None = None,
    sleep_s: float = 0.3,
):
    """
    Descarga un dataset Socrata (datos.gov.co) en páginas usando $limit/$offset.
    Devuelve un DataFrame con hasta max_rows filas (o menos si el dataset es menor).
    """
    headers = {}
    if app_token:
        headers["X-App-Token"] = app_token

    rows_collected = 0
    offset = 0
    frames = []

    session = requests.Session()

    while rows_collected < max_rows:
        params = {
            "$limit": min(limit, max_rows - rows_collected),
            "$offset": offset
        }
        if select: params["$select"] = select
        if where:  params["$where"] = where
        if order:  params["$order"] = order

        r = session.get(api_url, params=params, headers=headers, timeout=60)
        r.raise_for_status()
        batch = r.json()

        if not batch:
            break

        df_batch = pd.DataFrame(batch)
        frames.append(df_batch)

        n = len(df_batch)
        rows_collected += n
        offset += n

        print(f"Descargadas {rows_collected:,} filas...", end="\r")
        time.sleep(sleep_s)  # Respetar API

        # Si el lote vino más pequeño que 'limit', probablemente no hay más
        if n < params["$limit"]:
            break

    if not frames:
        return pd.DataFrame()

    df = pd.concat(frames, ignore_index=True)
    print(f"\n✅ Descarga completa: {len(df):,} filas.")
    return df

# Ejecutar descarga (ajusta max_rows si quieres más/menos)
df_raw = fetch_socrata_dataset(API_URL, max_rows=100_000, app_token=SODA_APP_TOKEN)
df_raw.to_parquet(RAW_FILE, index=False)
df_raw.head(3)


Descargadas 100,000 filas...
✅ Descarga completa: 100,000 filas.


,bancarizado,codigodepartamentoatencion,codigomunicipioatencion,discapacidad,estadobeneficiario,etnia,fechainscripcionbeneficiario,genero,nivelescolaridad,nombredepartamentoatencion,nombremunicipioatencion,pais,tipoasignacionbeneficio,tipobeneficio,tipodocumento,tipopoblacion,rangobeneficioconsolidadoasignado,rangoultimobeneficioasignado,fechaultimobeneficioasignado,rangoedad,titular,cantidaddebeneficiarios
0,SI,08,08421,NO,ACTIVO,AFROCOLOMBIANO – NEGRO,2012-12-01,Hombre,ND,ATLANTICO,LURUACO,ND,MONETARIO,ND,CC,UNIDOS,4.500.001 - 6.000.000,0 - 1.300.000,2018-01-01,30-49,SI,1
1,NO,13,13673,NO,NO ACTIVO,ND,2012-11-01,Mujer,ND,BOLIVAR,SANTA CATALINA,ND,ND,ND,TI,ND,0 - 1.500.000,0 - 1.300.000,1900-01-01,18-29,NO,21
2,SI,08,08421,NO,ACTIVO,AFROCOLOMBIANO – NEGRO,2012-12-01,Hombre,ND,ATLANTICO,LURUACO,ND,MONETARIO,ND,CC,UNIDOS,4.500.001 - 6.000.000,0 - 1.300.000,2018-01-01,30-49,SI,1


In [15]:
print("Shape:", df_raw.shape)
print("\nColumnas:\n", list(df_raw.columns))

print("\nTipos inferidos por pandas:\n")
display(df_raw.convert_dtypes().dtypes)

# Muestra de registros
display(df_raw.head(10))

# % de nulos por columna
nulls = (df_raw.isna().mean().sort_values(ascending=False) * 100).round(2)
display(nulls.to_frame("pct_nulls").head(20))


Shape: (100000, 22)

Columnas:
 ['bancarizado', 'codigodepartamentoatencion', 'codigomunicipioatencion', 'discapacidad', 'estadobeneficiario', 'etnia', 'fechainscripcionbeneficiario', 'genero', 'nivelescolaridad', 'nombredepartamentoatencion', 'nombremunicipioatencion', 'pais', 'tipoasignacionbeneficio', 'tipobeneficio', 'tipodocumento', 'tipopoblacion', 'rangobeneficioconsolidadoasignado', 'rangoultimobeneficioasignado', 'fechaultimobeneficioasignado', 'rangoedad', 'titular', 'cantidaddebeneficiarios']

Tipos inferidos por pandas:



bancarizado                          string[python]
codigodepartamentoatencion           string[python]
codigomunicipioatencion              string[python]
discapacidad                         string[python]
estadobeneficiario                   string[python]
etnia                                string[python]
fechainscripcionbeneficiario         string[python]
genero                               string[python]
nivelescolaridad                     string[python]
nombredepartamentoatencion           string[python]
nombremunicipioatencion              string[python]
pais                                 string[python]
tipoasignacionbeneficio              string[python]
tipobeneficio                        string[python]
tipodocumento                        string[python]
tipopoblacion                        string[python]
rangobeneficioconsolidadoasignado    string[python]
rangoultimobeneficioasignado         string[python]
fechaultimobeneficioasignado         string[python]
rangoedad   

,bancarizado,codigodepartamentoatencion,codigomunicipioatencion,discapacidad,estadobeneficiario,etnia,fechainscripcionbeneficiario,genero,nivelescolaridad,nombredepartamentoatencion,nombremunicipioatencion,pais,tipoasignacionbeneficio,tipobeneficio,tipodocumento,tipopoblacion,rangobeneficioconsolidadoasignado,rangoultimobeneficioasignado,fechaultimobeneficioasignado,rangoedad,titular,cantidaddebeneficiarios
0,SI,08,08421,NO,ACTIVO,AFROCOLOMBIANO – NEGRO,2012-12-01,Hombre,ND,ATLANTICO,LURUACO,ND,MONETARIO,ND,CC,UNIDOS,4.500.001 - 6.000.000,0 - 1.300.000,2018-01-01,30-49,SI,1
1,NO,13,13673,NO,NO ACTIVO,ND,2012-11-01,Mujer,ND,BOLIVAR,SANTA CATALINA,ND,ND,ND,TI,ND,0 - 1.500.000,0 - 1.300.000,1900-01-01,18-29,NO,21
2,SI,08,08421,NO,ACTIVO,AFROCOLOMBIANO – NEGRO,2012-12-01,Hombre,ND,ATLANTICO,LURUACO,ND,MONETARIO,ND,CC,UNIDOS,4.500.001 - 6.000.000,0 - 1.300.000,2018-01-01,30-49,SI,1
3,SI,08,08421,NO,ACTIVO,AFROCOLOMBIANO – NEGRO,2012-12-01,Hombre,ND,ATLANTICO,LURUACO,ND,MONETARIO,ND,CC,UNIDOS,4.500.001 - 6.000.000,0 - 1.300.000,2018-01-01,30-49,SI,1
4,SI,08,08421,NO,ACTIVO,AFROCOLOMBIANO – NEGRO,2012-12-01,Hombre,ND,ATLANTICO,LURUACO,ND,MONETARIO,ND,CC,UNIDOS,4.500.001 - 6.000.000,0 - 1.300.000,2018-01-01,30-49,SI,1
5,NO,41,41770,NO,ACTIVO,ND,2012-11-01,Hombre,PRIMARIA,HUILA,SUAZA,ND,ND,EDUCACIÓN PRIMARIANUTRICIÓN MENOR,RC,SISBEN,0 - 1.500.000,0 - 1.300.000,1900-01-01,06-17,NO,2
6,ND,47,47318,NO,ACTIVO,ND,2012-12-01,Mujer,ND,MAGDALENA,GUAMAL,Colombia,MONETARIO,ND,CC,SISBEN,> 6.000.001,0 - 1.300.000,2018-01-01,30-49,SI,1
7,ND,73,73217,NO,ACTIVO,ND,2013-02-01,Mujer,ND,TOLIMA,COYAIMA,ND,ND,NUTRICIÓN MENOR,No Definido,INDIGENAS,0 - 1.500.000,0 - 1.300.000,1900-01-01,06-17,NO,1
8,NO,95,95001,NO,ACTIVO,ND,2014-05-01,Hombre,ND,GUAVIARE,SAN JOSE DEL GUAVIARE,ND,ND,NUTRICIÓN MENOR,RC,DESPLAZADOS,0 - 1.500.000,0 - 1.300.000,1900-01-01,06-17,NO,5
9,NO,85,85001,NO,ACTIVO,ND,2012-10-01,Mujer,ND,CASANARE,YOPAL,ND,MONETARIO,ND,CC,SISBEN,1.500.001 - 3.000.000,0 - 1.300.000,2018-01-01,50-65,SI,4


,pct_nulls
bancarizado,1.86
codigodepartamentoatencion,0.00
codigomunicipioatencion,0.00
discapacidad,0.00
estadobeneficiario,0.00
etnia,0.00
fechainscripcionbeneficiario,0.00
genero,0.00
nivelescolaridad,0.00
nombredepartamentoatencion,0.00


In [16]:
def standardize_columns(columns):
    out = []
    for c in columns:
        c2 = (
            str(c)
            .strip()
            .lower()
            .replace(" ", "_")
            .replace("-", "_")
            .replace("/", "_")
        )
        c2 = re.sub(r"[^a-z0-9_]", "", c2)
        c2 = re.sub(r"_+", "_", c2).strip("_")
        out.append(c2)
    return out

def is_mostly_numeric(series, threshold=0.8):
    s = series.dropna().astype(str)
    if len(s) == 0:
        return False
    numeric_like = s.str.match(r"^-?\d+(\.\d+)?$")
    return numeric_like.mean() >= threshold

def parse_candidate_dates(df):
    date_patterns = ("fecha", "date", "fec_", "_fec", "fech")
    for col in df.columns:
        if any(pat in col for pat in date_patterns):
            try:
                df[col] = pd.to_datetime(df[col], errors="coerce", utc=False)
            except Exception:
                pass
    return df

def cast_numeric_columns(df):
    for col in df.columns:
        if df[col].dtype == "object" and is_mostly_numeric(df[col], threshold=0.7):
            df[col] = pd.to_numeric(df[col], errors="coerce")
    return df

def basic_clean(df):
    df = df.copy()
    df.columns = standardize_columns(df.columns)
    # Elimina duplicados exactos (si aplica)
    df = df.drop_duplicates()
    # Casts
    df = cast_numeric_columns(df)
    df = parse_candidate_dates(df)
    return df

df = basic_clean(df_raw)
df.to_parquet(PROC_FILE, index=False)
df.info()


<class 'pandas.core.frame.DataFrame'>
Index: 48679 entries, 0 to 99999
Data columns (total 22 columns):
 #   Column                             Non-Null Count  Dtype         
---  ------                             --------------  -----         
 0   bancarizado                        46850 non-null  object        
 1   codigodepartamentoatencion         48679 non-null  int64         
 2   codigomunicipioatencion            48679 non-null  int64         
 3   discapacidad                       48679 non-null  object        
 4   estadobeneficiario                 48679 non-null  object        
 5   etnia                              48679 non-null  object        
 6   fechainscripcionbeneficiario       48679 non-null  datetime64[ns]
 7   genero                             48679 non-null  object        
 8   nivelescolaridad                   48679 non-null  object        
 9   nombredepartamentoatencion         48679 non-null  object        
 10  nombremunicipioatencion            4867

In [17]:
def top_counts_for_categoricals(df, max_unique=50, topn=10):
    stats = {}
    for col in df.columns:
        nunique = df[col].nunique(dropna=True)
        if df[col].dtype == "object" and 1 <= nunique <= max_unique:
            vc = df[col].value_counts(dropna=False).head(topn)
            stats[col] = vc
    return stats

cat_stats = top_counts_for_categoricals(df, max_unique=80, topn=10)
for col, vc in cat_stats.items():
    print(f"\n== {col} ==")
    display(vc)

if not cat_stats:
    print("No se detectaron columnas categóricas de baja cardinalidad.")



== bancarizado ==


bancarizado
NO     26400
ND     12972
SI      7478
NaN     1829
Name: count, dtype: int64


== discapacidad ==


discapacidad
NO    39843
ND     8132
SI      704
Name: count, dtype: int64


== estadobeneficiario ==


...

rangoultimobeneficioasignado
0 - 1.300.000            48677
1.300.001 - 2.600.000        2
Name: count, dtype: int64


== rangoedad ==


rangoedad
06-17    19039
18-29    13375
30-49     9261
50-65     5027
>65       1977
Name: count, dtype: int64


== titular ==


titular
NO    31382
SI    17297
Name: count, dtype: int64

In [18]:
# Exporta versión limpia a CSV (para Power BI/Looker Studio)
CSV_OUT = PROC_DIR / "properidadsocial_clean.csv"
df.to_csv(CSV_OUT, index=False, encoding="utf-8")
print("Archivos listos:\n -", PROC_FILE, "\n -", CSV_OUT)

Archivos listos:
 - data\processed\xfif_myr2_clean.parquet 
 - data\processed\properidadsocial_clean.csv
